In [1]:
import glob
exp = "09-30_18-53__con001_conScale1_intraLoss_HT21_5e-05"
gpu = 5

files = glob.glob(f"/nfs/home/leo0511/Research/People-Flows/exp/HT21/FT/{exp}/ep*")
for file in files:
    print(f"python test_HT21.py --model_path {file} --GPU_ID {gpu}")

python test_HT21.py --model_path /nfs/home/leo0511/Research/People-Flows/exp/HT21/FT/09-30_18-53__con001_conScale1_intraLoss_HT21_5e-05/ep_5_iter_11000_mae_3.052_mse_3.716_seq_MAE_23.008_WRAE_35.397_MIAE_3.083_MOAE_5.008.pth --GPU_ID 5
python test_HT21.py --model_path /nfs/home/leo0511/Research/People-Flows/exp/HT21/FT/09-30_18-53__con001_conScale1_intraLoss_HT21_5e-05/ep_10_iter_24000_mae_11.005_mse_11.698_seq_MAE_6.883_WRAE_10.588_MIAE_2.353_MOAE_4.266.pth --GPU_ID 5
python test_HT21.py --model_path /nfs/home/leo0511/Research/People-Flows/exp/HT21/FT/09-30_18-53__con001_conScale1_intraLoss_HT21_5e-05/ep_17_iter_44000_mae_12.091_mse_12.646_seq_MAE_1.325_WRAE_2.039_MIAE_1.645_MOAE_3.939.pth --GPU_ID 5
python test_HT21.py --model_path /nfs/home/leo0511/Research/People-Flows/exp/HT21/FT/09-30_18-53__con001_conScale1_intraLoss_HT21_5e-05/ep_10_iter_25000_mae_23.206_mse_23.506_seq_MAE_24.260_WRAE_37.322_MIAE_1.120_MOAE_1.703.pth --GPU_ID 5
python test_HT21.py --model_path /nfs/home/leo0511